# Pizza pizza pizza

Get top venues by keyword and location, querying Google, Yelp, Foursquare

#### Motivation:
 - Gmaps and Yelp GUIs sorting and filtering are limited.
 - Query all 3 services and bring combined results into sortable table and map widget

#### Google

 - Needs a Google API key and gmaps python module
 - [Create Google Cloud credentials and give access to Places APIs](https://console.cloud.google.com/google/maps-apis/credentials) (also, restrict IP or set other restrictions)
 - `conda install -c conda-forge -y gmaps`
 - put key in `apikey.txt`
 - `gmaps` Jupyter nbextension to show maps in notebook, with marker pins etc.

```
conda install -c conda-forge -y jupyter_contrib_nbextensions
jupyter nbextension enable --py gmaps
jupyter notebook
```

#### Yelp
 - needs Yelp API key and module
 - https://www.yelp.com/developers/documentation/v3
 - https://github.com/gfairchild/yelpapi
 - put key in `yelpkey.txt`
 
#### Foursquare
- Needs Foursquare API key and module
- https://developer.foursquare.com/docs/places-api/getting-started/
- https://github.com/mLewisLogic/foursquare
- OAuth id in `foursquare_id.txt`
- OAuth secret in `foursquare_secret.txt`

See `requirements.txt` for versions used, other requirements (requests, beakerx, folium, Flask)

Was going to try OpenTable and TripAdvisor but their language seems to limit API key access to approved commercial partners.
- https://dev.opentable.com/affiliate-partners/
- https://www.tripadvisor.com/APIAccessSupport

In [1]:
import time
from pprint import pprint
import ipywidgets
from ipywidgets import widgets, interact
from itertools import product
from os import path

import traceback
import pdb

import numpy as np
import pandas as pd

# cluster similar entities
import pandas_dedupe

import sklearn
from sklearn.preprocessing import StandardScaler

import requests, json 

import qgrid
from qgrid import show_grid

# convert coords to km using haversine distance 
import geopy
from geopy.distance import distance

import gmaps
with open('secrets/apikey.txt') as f:
    api_key = f.readline().strip()
    f.close
gmaps.configure(api_key=api_key)

# https://github.com/gfairchild/yelpapi
import yelpapi
from yelpapi import YelpAPI
with open('secrets/yelpkey.txt') as f:
    yelp_key = f.readline().strip()
    f.close
yelp_api = YelpAPI(yelp_key)

import foursquare
from foursquare import Foursquare, FoursquareException
with open('secrets/foursquare_id.txt') as f:
    foursquare_id = f.readline().strip()
    f.close
with open('secrets/foursquare_secret.txt') as f:
    foursquare_secret = f.readline().strip()
    f.close

# interactive maps
import folium

print("%-20s %s" % ("numpy", np.__version__))
print("%-20s %s" % ("pandas", pd.__version__))
print("%-20s %s" % ("ipywidgets", ipywidgets.__version__))
print("%-20s %s" % ("qgrid", qgrid.__version__))
print("%-20s %s" % ("sklearn", sklearn.__version__))
print("%-20s %s" % ("requests", requests.__version__))
print("%-20s %s" % ("geopy", geopy.__version__))
print("%-20s %s" % ("gmaps", gmaps.__version__))
print("%-20s %s" % ("foursquare", foursquare.__version__))
print("%-20s %s" % ("folium", folium.__version__))


numpy                1.18.1
pandas               1.0.3
ipywidgets           7.5.1
qgrid                1.3.1
sklearn              0.22.1
requests             2.23.0
geopy                2.0.0
gmaps                0.9.0
foursquare           1!2020.1.30
folium               0.11.0


## Google Maps

In [2]:
# pick a search term
keyword_options = [('Pizza', 'pizza'), ('Coffee', 'coffee'), ('Ice Cream', 'icecream')]
keyword = 'pizza'

@interact
def get_kw(kw = widgets.Dropdown(
    options=keyword_options,
    value=keyword,
    description='Search term:  ',
    disabled=False,
)):
    global keyword
    keyword = kw
    return None


interactive(children=(Dropdown(description='Search term:  ', options=(('Pizza', 'pizza'), ('Coffee', 'coffee')…

In [3]:
# pick a location
location = '40.7484, -73.9857'
location_coords = tuple(eval(location))
location_options = [('Midtown', '40.7484, -73.9857'),
                    ('Downtown', '40.7077443,-74.0139089'),
                    ('Upper East Side', '40.7711473,-73.9661166'),
                    ('Upper West Side', '40.778794,-73.984257'),
                    ('Brooklyn Heights', '40.6915812,-73.9954095'), 
                    ('Grand Army Plaza', '40.671872,-73.972544'),
                    ('Bay Ridge', '40.6292633,-74.0309554'),
                    ('Williamsburg', '40.7144609,-73.9553373'),
                  ]

@interact
def get_loc(loc = widgets.Dropdown(
    options=location_options,
    value=location,
    description='Location:',
)):
    global location
    global location_coords
    location = loc
    location_coords = tuple(eval(loc))
    return None

interactive(children=(Dropdown(description='Location:', options=(('Midtown', '40.7484, -73.9857'), ('Downtown'…

In [4]:
# pick anything we don't have a pickle file for
for test_keyword, test_location in product(keyword_options, location_options):
    keyword = test_keyword[1]
    location_name = test_location[0]
    location = test_location[1]
    location_coords = tuple(eval(test_location[1]))
    test_pickle_filename = keyword + "_" + location_name.replace(' ', '').lower() + ".pkl"
    
    if not path.exists(test_pickle_filename):
        break
        
print(location_name, keyword, location, location_coords)
print(test_pickle_filename)

Downtown pizza 40.7077443,-74.0139089 (40.7077443, -74.0139089)
pizza_downtown.pkl


In [5]:
# gmap of chosen location
figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(center=location_coords, zoom_level=12, layout=figure_layout)
fig.add_layer(gmaps.marker_layer([location_coords]))
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [6]:
# global options for all search services APIs
MIN_USER_RATINGS = 20
MIN_RATING = 0
NRESULTS = 50
RADIUS = 1000


In [7]:
# gmaps options
GMAPS_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# https://developers.google.com/places/web-service/search#TextSearchRequests
#https://developers.google.com/places/web-service/supported_types
# rankby='prominence'
rankby='distance'
ltype='establishment'


In [8]:
def gmaps_get_first_page(api_key, location, **kwargs):
    """get first page of results from gmaps using api_key, location, kwargs for search spec"""
    # use either rankby or radius kwarg
    request_url = GMAPS_URL + '?key=' + api_key
    request_url += '&location=' + location
    for name, val in kwargs.items():
        request_url += '&' + name + '=' + str(val)
    r = requests.get(request_url)
    j = r.json()
    return j


def gmaps_get_next_page(api_key, next_page_token):
    """get next search engine results page page using search token, waiting until available"""
    r = requests.get(GMAPS_URL + '?pagetoken=' + next_page_token +
                        '&key=' + api_key)
    for i in range(10):
        j = r.json()
        if not j['results']: # wait for next page to be available
            time.sleep(5)
            continue
        else:
            return j


def gmaps_get_all_df(api_key, location, **kwargs):
    """return dataframe of all results using api_key, location, search kwargs"""
    # get first page
    j = gmaps_get_first_page(api_key, location, **kwargs)
    venues_df = pd.json_normalize(j['results'])

    # get pages while additional pages available
    while 'next_page_token' in j:
        next_page_token = j['next_page_token']
        time.sleep(5)
        j = gmaps_get_next_page(api_key, next_page_token)
        venues_df = venues_df.append(pd.json_normalize(j['results']))
        
    return venues_df


def gmaps_get_df(location_coords, keyword):

    # use either rankby or radius
    location = "%.7f,%.7f" % location_coords
    gmaps_df = gmaps_get_all_df(api_key, location, keyword=keyword, ltype=ltype, radius=RADIUS)
    if gmaps_df.empty:
        return None
    else:
        # gmaps_get_df(api_key, location, keyword=keyword, ltype=ltype, radius=RADIUS)
        gmaps_df = gmaps_df.loc[(gmaps_df['user_ratings_total'] >= MIN_USER_RATINGS) & (gmaps_df['rating'] >= MIN_RATING)] \
                           .sort_values(['rating', 'user_ratings_total'], ascending=False) \
                           .reset_index(drop=True)
        gmaps_df = gmaps_df[['name', 'vicinity', 'rating', 'user_ratings_total', 'geometry.location.lat', 'geometry.location.lng']]
        gmaps_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng']
        # drop trailing ", Brooklyn"
        gmaps_df['address'] = gmaps_df['address'].apply(lambda address: " ".join(address.split(',')[:-1]))
        gmaps_df['distance'] = gmaps_df.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
                                              axis=1)
        return gmaps_df


In [9]:
%%time
gmaps_df = gmaps_get_df(location_coords, keyword)
gmaps_df = gmaps_df.loc[(gmaps_df['nratings'] >= MIN_USER_RATINGS) & (gmaps_df['rating'] >= MIN_RATING)] \
        .sort_values(['rating', 'nratings'], ascending=False) \
        .reset_index(drop=True)
gmaps_df.to_pickle('gmaps_' + test_pickle_filename)
gmaps_df


CPU times: user 74.9 ms, sys: 8.27 ms, total: 83.1 ms
Wall time: 7.03 s


,name,address,rating,nratings,lat,lng,distance
0,Joe’s Pizza,124 Fulton St,4.6,981,40.710112,-74.007739,0.583948
1,Big Al's Chicago Style Pizza,9 Thames St,4.6,150,40.708912,-74.011595,0.234572
2,&pizza - Wall Street,63 Wall St,4.6,118,40.705765,-74.008547,0.503598
3,Kesté Wall Street,77 Fulton St,4.5,539,40.709245,-74.004893,0.779915
4,Famous Calabria Pizza,27 St James Pl,4.5,291,40.711942,-73.999347,1.315859
5,Benvenuti Pizzeria,235 South End Ave,4.5,126,40.709660,-74.016741,0.320179
6,Adrienne's Pizzabar,54 Stone St,4.4,1636,40.704294,-74.010065,0.502333
7,La Pizza & La Pasta,101 Liberty St 3rd floor,4.4,1037,40.710065,-74.011976,0.305094
8,Picasso Pizza,303 South End Ave,4.4,474,40.710385,-74.016469,0.364439
9,The Grotto Pizzeria,69 New St,4.4,342,40.705674,-74.012384,0.263531


In [10]:
# sort grid by clicking on header (can also click on filter button)
show_grid(gmaps_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [11]:
# plot on google map

markers = [(row.lat, row.lng) for row in gmaps_df.itertuples()]
marker_hover = ["%s: %s (%s)" % (row.name, row.rating, row.nratings) for row in gmaps_df.itertuples()]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Google rating</dt><dd>{rating}</dd>
<dt>Google reviews</dt><dd>{nratings}</dd>
</dl>
"""
marker_info = [info_box_template.format(**row) for i, row in gmaps_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=eval(location), zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [12]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
    
venues_map

## Yelp

In [13]:
def yelp_get_df(location_coords, keyword):
    lat, lng = location_coords
    response = yelp_api.search_query(categories=keyword, latitude=lat, longitude=lng, 
                                     radius=RADIUS, sort_by=rankby, limit=NRESULTS)

    yelp_df = pd.json_normalize(response['businesses'])
    if not yelp_df.empty:    
        yelp_df = yelp_df.loc[(yelp_df['review_count'] >= MIN_USER_RATINGS) & (yelp_df['rating'] >= MIN_RATING)] \
                         .sort_values(['rating', 'review_count'], ascending=False) \
                         .reset_index(drop=True)
        display_columns = ['name', 'location.address1', 'rating', 'review_count', 'coordinates.latitude', 'coordinates.longitude', 'url']
        yelp_df = yelp_df[display_columns]
        yelp_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng', 'url']
        yelp_df['distance'] = yelp_df.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
                                              axis=1)
        
        if yelp_df.empty:
            return None
        else:
            return yelp_df
    else:
        return None

yelp_df = yelp_get_df(location_coords, keyword)
yelp_df

,name,address,rating,nratings,lat,lng,url,distance
0,Scott's Pizza Tours,,5.0,314,40.750244,-73.997025,https://www.yelp.com/biz/scotts-pizza-tours-ne...,4.930385
1,&pizza,63 Wall St,4.5,81,40.705790,-74.008590,https://www.yelp.com/biz/and-pizza-new-york-13...,0.499134
2,Jiannetto's Pizza Truck,100 Wall St,4.5,31,40.705250,-74.006750,https://www.yelp.com/biz/jiannettos-pizza-truc...,0.665374
3,Adrienne's Pizzabar,54 Stone St,4.0,1656,40.704336,-74.010173,https://www.yelp.com/biz/adriennes-pizzabar-ne...,0.492842
4,Sauce & Barrel,97 Washington St,4.0,416,40.707999,-74.014265,https://www.yelp.com/biz/sauce-and-barrel-new-...,0.041304
5,Inatteso Pizzabar,28 West St,4.0,317,40.705657,-74.017206,https://www.yelp.com/biz/inatteso-pizzabar-new...,0.362404
6,Underground Pizza,3 Hanover Sq,4.0,294,40.704990,-74.009610,https://www.yelp.com/biz/underground-pizza-new...,0.474899
7,Stage Door Pizza,26 Vesey St,4.0,246,40.712005,-74.009656,https://www.yelp.com/biz/stage-door-pizza-new-...,0.594124
8,Joe's Pizza,124 Fulton St,4.0,200,40.710130,-74.007721,https://www.yelp.com/biz/joes-pizza-new-york-1...,0.586199
9,The Grotto Pizzeria & Ristorante,69 New St,4.0,143,40.705740,-74.012402,https://www.yelp.com/biz/the-grotto-pizzeria-a...,0.256428


In [14]:
yelp_df.to_pickle('yelp_' + test_pickle_filename)
show_grid(yelp_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [15]:
markers = [(row.lat, row.lng) for row in yelp_df.itertuples()]
marker_hover = ["%s: %s (%s)" % (row.name, row.rating, row.nratings) for row in yelp_df.itertuples()]

info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Yelp rating</dt><dd>{rating}</dd>
<dt>Yelp reviews</dt><dd>{nratings}</dd>
</dl>
"""

marker_info = [info_box_template.format(**row) for i, row in yelp_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=location_coords, zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [16]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map

## Foursquare

In [17]:
def foursquare_get_df(location_coords, keyword):
    location_str = "%.7f,%.7f" % location_coords
    client = Foursquare(client_id=foursquare_id, 
                                   client_secret=foursquare_secret, 
                                   redirect_uri='http://streeteye.com/')
    response = client.venues.search(params={'query': keyword, 'll': "%s" % location_str, 
                                            'radius': RADIUS, 'limit': NRESULTS})

    foursquare_array = []

    for i, venue in pd.json_normalize(response['venues']).iterrows():
        venue_id = venue['id']
        # query detailed venue info from foursquare
        venue_name = venue['name']
        venue_address = venue['location.address']
        # sometimes no URL
        try:
            venue_url = venue['delivery.url']
        except:
            venue_url = ''
        venue_lat = venue['location.lat']
        venue_lng = venue['location.lng']
        # default these to -1
        try:
            # get rating, nratings with another API call for venue details
            venue_details = client.venues(venue_id)['venue']
            venue_rating = venue_details['rating']
            venue_nratings = venue_details['ratingSignals']
        except FoursquareException as e:
            print("Foursquare exception", type(e), str(e))
        except Exception as e:
            continue
            # sometimes no rating ... probably not popular enough
            # print(type(e), str(e))
            # print(traceback.format_exc())
            # print("No rating for %s" % venue_name)

        foursquare_array.append([venue_name, venue_address, venue_rating, venue_nratings, venue_lat, venue_lng, venue_url])
            
    foursquare_df = pd.DataFrame(foursquare_array)
    
    if len(foursquare_df) and len(foursquare_df.columns):
        foursquare_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng', 'url']
        foursquare_df = foursquare_df.loc[(foursquare_df['nratings'] >= MIN_USER_RATINGS) & (foursquare_df['rating'] >= MIN_RATING)] \
                                     .sort_values(['rating', 'nratings'], ascending=False) \
                                     .reset_index(drop=True)
        
        foursquare_df['distance'] = foursquare_df.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
                                                        axis=1)
        if foursquare_df.empty:
            return None
        else:
            return foursquare_df
    else:
        return None

foursquare_df = foursquare_get_df(location_coords, keyword)
foursquare_df

,name,address,rating,nratings,lat,lng,url,distance
0,Joe’s Pizza,124 Fulton St,8.9,96,40.710318,-74.007694,https://www.seamless.com/menu/joes-pizza-124-f...,0.597910
1,Adrienne's Pizza Bar,54 Stone St,8.6,880,40.704348,-74.010223,NaN,0.489164
2,Harry's Italian Pizza Bar,2 Gold St,8.1,512,40.707634,-74.006996,NaN,0.584321
3,La Pizza & La Pasta,4 World Trade Ctr,8.0,270,40.710567,-74.012494,NaN,0.335443
4,Big Al's Chicago Style Pizza,9 Thames St,7.7,35,40.708892,-74.011562,https://www.seamless.com/menu/big-als-pizza-9-...,0.235798
5,Little Italy Pizza IV,50 Fulton St,7.4,40,40.708196,-74.004538,https://www.seamless.com/menu/little-italy-piz...,0.793444
6,Pizza Plus,4 South St,7.0,27,40.701693,-74.012907,https://www.seamless.com/menu/pizza-plus-4-sou...,0.677348
7,Harry's Italian Pizza Bar,225 Murray St,6.7,212,40.715218,-74.014739,NaN,0.832881
8,Underground Pizza,3 Hanover Sq,6.7,76,40.704900,-74.009797,https://www.seamless.com/menu/underground-pizz...,0.469562
9,Little Italy Gourmet Pizza,11 Park Pl,6.6,44,40.713044,-74.008334,https://www.seamless.com/menu/little-italy-gou...,0.753848


In [18]:
if foursquare_df is not None and not foursquare_df.empty:
    foursquare_df.to_pickle('foursquare_' + test_pickle_filename)
show_grid(foursquare_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [19]:
markers = [(row.lat, row.lng) for row in foursquare_df.itertuples()]
marker_hover = ["%s: %s (%s)" % (row.name, row.rating, row.nratings) for row in foursquare_df.itertuples()]

info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Foursquare rating</dt><dd>{rating}</dd>
<dt>Foursquare reviews</dt><dd>{nratings}</dd>
</dl>
"""
marker_info = [info_box_template.format(**d_item) for i, d_item in foursquare_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=location_coords, zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [20]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map

## Deduplicate and merge

In [21]:
# put everything in one big df
pd.set_option('display.max_rows', None)

gmaps_df_copy, yelp_df_copy, foursquare_df_copy = None, None, None

try:
    gmaps_df_copy = gmaps_df.copy()
    gmaps_df_copy['source'] = '0_gmaps'
except: 
    pass

try:
    yelp_df_copy = yelp_df.copy() 
    yelp_df_copy['source'] = '1_yelp'
except:
    pass

try:
    foursquare_df_copy = foursquare_df.copy()
    foursquare_df_copy['source'] = '2_foursquare'
except:
    pass


venues_df = pd.concat(list(filter(lambda df: df is not None, [gmaps_df_copy, yelp_df_copy, foursquare_df_copy]))).reset_index()
venues_df['latlong'] = venues_df[['lat','lng']].apply(tuple, axis=1)

venues_df.sort_values('name')

,index,name,address,rating,nratings,lat,lng,distance,source,url,latlong
35,1,&pizza,63 Wall St,4.5,81,40.705790,-74.008590,0.499134,1_yelp,https://www.yelp.com/biz/and-pizza-new-york-13...,"(40.70579, -74.00859)"
2,2,&pizza - Wall Street,63 Wall St,4.6,118,40.705765,-74.008547,0.503598,0_gmaps,NaN,"(40.7057647, -74.0085474)"
75,1,Adrienne's Pizza Bar,54 Stone St,8.6,880,40.704348,-74.010223,0.489164,2_foursquare,NaN,"(40.704348091685645, -74.01022261314714)"
37,3,Adrienne's Pizzabar,54 Stone St,4.0,1656,40.704336,-74.010173,0.492842,1_yelp,https://www.yelp.com/biz/adriennes-pizzabar-ne...,"(40.7043362, -74.0101733)"
6,6,Adrienne's Pizzabar,54 Stone St,4.4,1636,40.704294,-74.010065,0.502333,0_gmaps,NaN,"(40.7042941, -74.0100646)"
86,12,Amore's Pizza Restaurant,147 Chambers St,6.4,36,40.715860,-74.009888,0.963103,2_foursquare,https://www.seamless.com/menu/cafe-amores-pizz...,"(40.715859606149245, -74.00988835781403)"
18,18,Artichoke Basille's Pizza,70 Pine St,4.3,125,40.706333,-74.007319,0.578543,0_gmaps,NaN,"(40.7063334, -74.00731859999999)"
66,32,Artichoke Basille's Pizza,70 Pine St,3.0,62,40.706518,-74.007505,0.558042,1_yelp,https://www.yelp.com/biz/artichoke-basilles-pi...,"(40.706518, -74.007505)"
67,33,Benvenuti Pizzeria,235 S End Ave,3.0,50,40.709709,-74.016783,0.326472,1_yelp,https://www.yelp.com/biz/benvenuti-pizzeria-ne...,"(40.7097089406072, -74.0167829487317)"
5,5,Benvenuti Pizzeria,235 South End Ave,4.5,126,40.709660,-74.016741,0.320179,0_gmaps,NaN,"(40.7096597, -74.0167409)"


In [22]:
# manual fix 
# venues_df.loc[venues_df['address']=='212b Pacific St', 'address'] = '212 Pacific St'
# venues_df

In [23]:
# run dedupe algorithm using name, address as default texts, latlong as latlong
venues_df['shortname'] = venues_df['name'].apply(lambda n: n[:25])
venues_df2 = pandas_dedupe.dedupe_dataframe(venues_df, ['name', 'address', ('latlong', 'LatLong')])


Importing data ...
Reading from dedupe_dataframe_learned_settings
Clustering...
# duplicate sets 54


In [24]:
# view clustering
venues_df['cluster'] = venues_df2['cluster id']
venues_df = venues_df.sort_values(['cluster', 'source'])[['cluster', 'name', 'address', 'rating', 'nratings', 'lat', 'lng', 'distance', 'source']]
venues_df

,cluster,name,address,rating,nratings,lat,lng,distance,source
0,0,Joe’s Pizza,124 Fulton St,4.6,981,40.7101118,-74.0077386,0.5839478253839243,0_gmaps
42,0,Joe's Pizza,124 Fulton St,4.0,200,40.7101297729083,-74.0077206864953,0.5861990454141909,1_yelp
74,0,Joe’s Pizza,124 Fulton St,8.9,96,40.71031775378812,-74.00769390169111,0.5979099660311961,2_foursquare
1,1,Big Al's Chicago Style Pizza,9 Thames St,4.6,150,40.7089116,-74.01159539999999,0.2345723937191282,0_gmaps
58,1,Big Al's Chicago Style Pizza,9 Thames St,3.5,73,40.7089285616514,-74.0117729569844,0.22332575449201503,1_yelp
78,1,Big Al's Chicago Style Pizza,9 Thames St,7.7,35,40.708892320194174,-74.01156153941926,0.23579810787525093,2_foursquare
2,2,&pizza - Wall Street,63 Wall St,4.6,118,40.7057647,-74.0085474,0.5035979897455162,0_gmaps
35,2,&pizza,63 Wall St,4.5,81,40.70579,-74.00859,0.4991335187071133,1_yelp
5,3,Benvenuti Pizzeria,235 South End Ave,4.5,126,40.7096597,-74.0167409,0.3201790197976901,0_gmaps
67,3,Benvenuti Pizzeria,235 S End Ave,3.0,50,40.7097089406072,-74.0167829487317,0.326471995029614,1_yelp


In [25]:
# group by clusters
cluster_df = venues_df.groupby('cluster')[['name', 'address', 'lat', 'lng', 'distance', 'source']] \
    .first() \
    .reset_index()
cluster_df

,cluster,name,address,lat,lng,distance,source
0,0,Joe’s Pizza,124 Fulton St,40.7101118,-74.0077386,0.5839478253839243,0_gmaps
1,1,Big Al's Chicago Style Pizza,9 Thames St,40.7089116,-74.01159539999999,0.2345723937191282,0_gmaps
2,2,&pizza - Wall Street,63 Wall St,40.7057647,-74.0085474,0.5035979897455162,0_gmaps
3,3,Benvenuti Pizzeria,235 South End Ave,40.7096597,-74.0167409,0.3201790197976901,0_gmaps
4,4,Adrienne's Pizzabar,54 Stone St,40.7042941,-74.0100646,0.502332680670826,0_gmaps
5,5,Picasso Pizza,303 South End Ave,40.7103852,-74.0164692,0.36443871916847875,0_gmaps
6,6,The Grotto Pizzeria,69 New St,40.7056743,-74.0123839,0.263530757401171,0_gmaps
7,7,Rosella's Pizzeria,164 William Street #2611 between Ann St & Bee...,40.7100678,-74.005622,0.7463004344991966,0_gmaps
8,8,Little Italy Pizza,11 Park Pl,40.713044,-74.00833399999999,0.7538480915927681,0_gmaps
9,9,Famous Famiglia Pizza,83 Maiden Ln,40.7075142,-74.00733199999999,0.556373690166844,0_gmaps


In [26]:
# make markers on clusters
# add ratings on all rows

markers = [(float(row.lat), float(row.lng)) for row in cluster_df.itertuples()]
marker_hover = ["%s" % (row.name) for row in cluster_df.itertuples()]

# make a dict by cluster, initialize rating string to ''
marker_dict = {i: {'name': row['name'],
                   'address': row['address'],
                   'lat': row['lat'],
                   'lng': row['lng'],
                   'rate_str': '',
                  } for i, row in cluster_df.iterrows()}

# add all rating strings
for i, row in venues_df.iterrows():
    cluster = row['cluster']
    if row.source[2:] == 'gmaps':
        marker_dict[cluster]['rate_str'] += "<dt>Google rating</dt><dd>%s (%s reviews)</dd>\n" % (row.rating, row.nratings)
    elif row.source[2:] == 'yelp':
        marker_dict[cluster]['rate_str'] += "<dt>Yelp rating</dt><dd>%s (%s reviews)</dd>\n" % (row.rating, row.nratings)
    elif row.source[2:] == 'foursquare':
        marker_dict[cluster]['rate_str'] += "<dt>Foursquare rating</dt><dd>%s (%s reviews)</dd>\n" % (row.rating, row.nratings)
    
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
{rate_str}

</dl>
"""    

marker_info = [info_box_template.format(**d_item) for i, d_item in marker_dict.items()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=location_coords, zoom_level=14)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [27]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map


In [28]:
venues_df

,cluster,name,address,rating,nratings,lat,lng,distance,source
0,0,Joe’s Pizza,124 Fulton St,4.6,981,40.7101118,-74.0077386,0.5839478253839243,0_gmaps
42,0,Joe's Pizza,124 Fulton St,4.0,200,40.7101297729083,-74.0077206864953,0.5861990454141909,1_yelp
74,0,Joe’s Pizza,124 Fulton St,8.9,96,40.71031775378812,-74.00769390169111,0.5979099660311961,2_foursquare
1,1,Big Al's Chicago Style Pizza,9 Thames St,4.6,150,40.7089116,-74.01159539999999,0.2345723937191282,0_gmaps
58,1,Big Al's Chicago Style Pizza,9 Thames St,3.5,73,40.7089285616514,-74.0117729569844,0.22332575449201503,1_yelp
78,1,Big Al's Chicago Style Pizza,9 Thames St,7.7,35,40.708892320194174,-74.01156153941926,0.23579810787525093,2_foursquare
2,2,&pizza - Wall Street,63 Wall St,4.6,118,40.7057647,-74.0085474,0.5035979897455162,0_gmaps
35,2,&pizza,63 Wall St,4.5,81,40.70579,-74.00859,0.4991335187071133,1_yelp
5,3,Benvenuti Pizzeria,235 South End Ave,4.5,126,40.7096597,-74.0167409,0.3201790197976901,0_gmaps
67,3,Benvenuti Pizzeria,235 S End Ave,3.0,50,40.7097089406072,-74.0167829487317,0.326471995029614,1_yelp


In [29]:
merge_df = cluster_df \
    .merge(venues_df.loc[venues_df['source']=='0_gmaps'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
    .rename(columns={'rating': 'gmaps_rating', 'nratings': 'gmaps_nratings'})
merge_df['gmaps_rating_std'] = StandardScaler().fit_transform(merge_df[['gmaps_rating']])

merge_df = merge_df \
    .merge(venues_df.loc[venues_df['source']=='1_yelp'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
    .rename(columns={'rating': 'yelp_rating', 'nratings': 'yelp_nratings'})
merge_df['yelp_rating_std'] = StandardScaler().fit_transform(merge_df[['yelp_rating']])

merge_df = merge_df \
    .merge(venues_df.loc[venues_df['source']=='2_foursquare'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
    .rename(columns={'rating': 'foursquare_rating', 'nratings': 'foursquare_nratings'})
merge_df['foursquare_rating_std'] = StandardScaler().fit_transform(merge_df[['foursquare_rating']])

merge_df 

,cluster,name,address,lat,lng,distance,source,gmaps_rating,gmaps_nratings,gmaps_rating_std,yelp_rating,yelp_nratings,yelp_rating_std,foursquare_rating,foursquare_nratings,foursquare_rating_std
0,0,Joe’s Pizza,124 Fulton St,40.7101118,-74.0077386,0.5839478253839243,0_gmaps,4.6,981,1.229911,4.0,200,0.858037,8.9,96,2.003460
1,1,Big Al's Chicago Style Pizza,9 Thames St,40.7089116,-74.01159539999999,0.2345723937191282,0_gmaps,4.6,150,1.229911,3.5,73,0.040859,7.7,35,0.753595
2,2,&pizza - Wall Street,63 Wall St,40.7057647,-74.0085474,0.5035979897455162,0_gmaps,4.6,118,1.229911,4.5,81,1.675215,NaN,NaN,NaN
3,3,Benvenuti Pizzeria,235 South End Ave,40.7096597,-74.0167409,0.3201790197976901,0_gmaps,4.5,126,0.905748,3.0,50,-0.776319,NaN,NaN,NaN
4,4,Adrienne's Pizzabar,54 Stone St,40.7042941,-74.0100646,0.502332680670826,0_gmaps,4.4,1636,0.581586,4.0,1656,0.858037,8.6,880,1.690994
5,5,Picasso Pizza,303 South End Ave,40.7103852,-74.0164692,0.36443871916847875,0_gmaps,4.4,474,0.581586,3.0,119,-0.776319,NaN,NaN,NaN
6,6,The Grotto Pizzeria,69 New St,40.7056743,-74.0123839,0.263530757401171,0_gmaps,4.4,342,0.581586,4.0,143,0.858037,NaN,NaN,NaN
7,7,Rosella's Pizzeria,164 William Street #2611 between Ann St & Bee...,40.7100678,-74.005622,0.7463004344991966,0_gmaps,4.3,519,0.257423,3.5,234,0.040859,NaN,NaN,NaN
8,8,Little Italy Pizza,11 Park Pl,40.713044,-74.00833399999999,0.7538480915927681,0_gmaps,4.3,359,0.257423,3.5,88,0.040859,6.6,44,-0.392114
9,9,Famous Famiglia Pizza,83 Maiden Ln,40.7075142,-74.00733199999999,0.556373690166844,0_gmaps,4.3,354,0.257423,4.0,38,0.858037,NaN,NaN,NaN


In [30]:
# simple average score
merge_df['meanrating'] = np.nanmean(merge_df[['gmaps_rating_std', 'yelp_rating_std', 'foursquare_rating_std']], axis=1)
merge_df.sort_values('meanrating', ascending=False)[['name', 'address', 'gmaps_rating', 'yelp_rating', 'foursquare_rating', 'meanrating']]


,name,address,gmaps_rating,yelp_rating,foursquare_rating,meanrating
38,Scott's Pizza Tours,,NaN,5.0,NaN,2.492392
39,Jiannetto's Pizza Truck,100 Wall St,NaN,4.5,NaN,1.675215
2,&pizza - Wall Street,63 Wall St,4.6,4.5,NaN,1.452563
0,Joe’s Pizza,124 Fulton St,4.6,4.0,8.9,1.363802
4,Adrienne's Pizzabar,54 Stone St,4.4,4.0,8.6,1.043539
28,Kesté Wall Street,77 Fulton St,4.5,NaN,NaN,0.905748
29,Famous Calabria Pizza,27 St James Pl,4.5,NaN,NaN,0.905748
41,Inatteso Pizzabar,28 West St,NaN,4.0,NaN,0.858037
40,Sauce & Barrel,97 Washington St,NaN,4.0,NaN,0.858037
42,Friendly Gourmet Pizza,59 Nassau St,NaN,4.0,NaN,0.858037


In [31]:
# bayes score
rating_cols = ['gmaps_rating_std', 'yelp_rating_std', 'foursquare_rating_std']
merge_df['nratings'] = merge_df[rating_cols].count(axis=1)
nratings_mean = np.mean(merge_df['nratings'])
rating_avg = np.nanmean(merge_df[rating_cols])
merge_df['w'] = merge_df['nratings']/(merge_df['nratings'] + nratings_mean)
merge_df['R'] = np.mean(merge_df[rating_cols], axis=1)
print('mean number of ratings', nratings_mean)
print('average rating', rating_avg)
merge_df['bayes_score'] = merge_df['w'] * merge_df['R'] + (1 - merge_df['w']) * rating_avg

show_grid(merge_df.sort_values('bayes_score', ascending=False)[['name', 'address', 'distance', 'gmaps_rating', 'yelp_rating', 'foursquare_rating', 'nratings', 'bayes_score']])

mean number of ratings 1.6851851851851851
average rating -1.7568364345716762e-16


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [32]:
def dedupe(dedupe_list):

    for i, source_df in enumerate(dedupe_list):
        source_df['source'] = i
    venues_df = pd.concat(dedupe_list).reset_index()
    venues_df['latlong'] = venues_df[['lat','lng']].apply(tuple, axis=1)
    venues_df['shortname'] = venues_df['name'].apply(lambda n: n[:25])

    # dedupe and assign cluster id
    venues_df2 = pandas_dedupe.dedupe_dataframe(venues_df, ['shortname', 'address', ('latlong', 'LatLong')])
    venues_df['cluster'] = venues_df2['cluster id']
    venues_df = venues_df.sort_values(['cluster', 'source'])[['cluster', 'name', 'address', 'rating', 'nratings', 'lat', 'lng', 'distance', 'source']]

    # group by clusters, uniquify name
    cluster_df = venues_df.groupby('cluster')[['name', 'address', 'lat', 'lng', 'distance', 'source']] \
                          .first() \
                          .reset_index()

    # merge ratings by source
    merge_df = cluster_df \
        .merge(venues_df.loc[venues_df['source']=='0'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
        .rename(columns={'rating': 'gmaps_rating', 'nratings': 'gmaps_nratings'})
    merge_df['gmaps_rating_std'] = StandardScaler().fit_transform(merge_df[['gmaps_rating']])

    merge_df = merge_df \
        .merge(venues_df.loc[venues_df['source']=='1'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
        .rename(columns={'rating': 'yelp_rating', 'nratings': 'yelp_nratings'})
    merge_df['yelp_rating_std'] = StandardScaler().fit_transform(merge_df[['yelp_rating']])

    merge_df = merge_df \
        .merge(venues_df.loc[venues_df['source']=='2'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
        .rename(columns={'rating': 'foursquare_rating', 'nratings': 'foursquare_nratings'})
    merge_df['foursquare_rating_std'] = StandardScaler().fit_transform(merge_df[['foursquare_rating']])

    # bayes score
    rating_cols = ['gmaps_rating_std', 'yelp_rating_std', 'foursquare_rating_std']
    merge_df['nratings'] = merge_df[rating_cols].count(axis=1)
    nratings_mean = np.mean(merge_df['nratings'])
    rating_avg = np.nanmean(merge_df[rating_cols])
    merge_df['w'] = merge_df['nratings']/(merge_df['nratings'] + nratings_mean)
    merge_df['R'] = np.mean(merge_df[rating_cols], axis=1)
    merge_df['bayes_score'] = merge_df['w'] * merge_df['R'] + (1 - merge_df['w']) * rating_avg
    merge_df = merge_df.sort_values('bayes_score', ascending=False)
    return merge_df

    
dedupe_list = list(filter(lambda df: df is not None, [gmaps_df, yelp_df, foursquare_df]))

dedupe_df = dedupe(dedupe_list)
dedupe_df


Importing data ...
Reading from dedupe_dataframe_learned_settings
Clustering...
# duplicate sets 54


,cluster,name,address,lat,lng,distance,source,gmaps_rating,gmaps_nratings,gmaps_rating_std,yelp_rating,yelp_nratings,yelp_rating_std,foursquare_rating,foursquare_nratings,foursquare_rating_std,nratings,w,R,bayes_score
38,38,Scott's Pizza Tours,,40.750244140625,-73.9970245361328,4.9303850451248294,1,NaN,NaN,NaN,5.0,314,2.492392,NaN,NaN,NaN,1,0.372414,2.492392,0.928201
0,0,Joe’s Pizza,124 Fulton St,40.7101118,-74.0077386,0.5839478253839243,0,4.6,981,1.229911,4.0,200,0.858037,8.9,96,2.003460,3,0.640316,1.363802,0.873265
2,2,&pizza - Wall Street,63 Wall St,40.7057647,-74.0085474,0.5035979897455162,0,4.6,118,1.229911,4.5,81,1.675215,NaN,NaN,NaN,2,0.542714,1.452563,0.788325
4,4,Adrienne's Pizzabar,54 Stone St,40.7042941,-74.0100646,0.502332680670826,0,4.4,1636,0.581586,4.0,1656,0.858037,8.6,880,1.690994,3,0.640316,1.043539,0.668195
39,39,Jiannetto's Pizza Truck,100 Wall St,40.70525,-74.00675,0.6653736322792096,1,NaN,NaN,NaN,4.5,31,1.675215,NaN,NaN,NaN,1,0.372414,1.675215,0.623873
1,1,Big Al's Chicago Style Pizza,9 Thames St,40.7089116,-74.01159539999999,0.2345723937191282,0,4.6,150,1.229911,3.5,73,0.040859,7.7,35,0.753595,3,0.640316,0.674788,0.432078
6,6,The Grotto Pizzeria,69 New St,40.7056743,-74.0123839,0.263530757401171,0,4.4,342,0.581586,4.0,143,0.858037,NaN,NaN,NaN,2,0.542714,0.719811,0.390651
29,29,Famous Calabria Pizza,27 St James Pl,40.7119418,-73.99934689999999,1.3158593427106806,0,4.5,291,0.905748,NaN,NaN,NaN,NaN,NaN,NaN,1,0.372414,0.905748,0.337313
28,28,Kesté Wall Street,77 Fulton St,40.7092446,-74.0048927,0.7799148904100284,0,4.5,539,0.905748,NaN,NaN,NaN,NaN,NaN,NaN,1,0.372414,0.905748,0.337313
25,25,Harry's Italian,2 Gold St,40.707555318913,-74.006898168681,0.5928193022318851,1,NaN,NaN,NaN,3.5,580,0.040859,8.1,512,1.170217,2,0.542714,0.605538,0.328634


In [33]:
dedupe_df.to_pickle(test_pickle_filename)
pd.read_pickle(test_pickle_filename)

,cluster,name,address,lat,lng,distance,source,gmaps_rating,gmaps_nratings,gmaps_rating_std,yelp_rating,yelp_nratings,yelp_rating_std,foursquare_rating,foursquare_nratings,foursquare_rating_std,nratings,w,R,bayes_score
38,38,Scott's Pizza Tours,,40.750244140625,-73.9970245361328,4.9303850451248294,1,NaN,NaN,NaN,5.0,314,2.492392,NaN,NaN,NaN,1,0.372414,2.492392,0.928201
0,0,Joe’s Pizza,124 Fulton St,40.7101118,-74.0077386,0.5839478253839243,0,4.6,981,1.229911,4.0,200,0.858037,8.9,96,2.003460,3,0.640316,1.363802,0.873265
2,2,&pizza - Wall Street,63 Wall St,40.7057647,-74.0085474,0.5035979897455162,0,4.6,118,1.229911,4.5,81,1.675215,NaN,NaN,NaN,2,0.542714,1.452563,0.788325
4,4,Adrienne's Pizzabar,54 Stone St,40.7042941,-74.0100646,0.502332680670826,0,4.4,1636,0.581586,4.0,1656,0.858037,8.6,880,1.690994,3,0.640316,1.043539,0.668195
39,39,Jiannetto's Pizza Truck,100 Wall St,40.70525,-74.00675,0.6653736322792096,1,NaN,NaN,NaN,4.5,31,1.675215,NaN,NaN,NaN,1,0.372414,1.675215,0.623873
1,1,Big Al's Chicago Style Pizza,9 Thames St,40.7089116,-74.01159539999999,0.2345723937191282,0,4.6,150,1.229911,3.5,73,0.040859,7.7,35,0.753595,3,0.640316,0.674788,0.432078
6,6,The Grotto Pizzeria,69 New St,40.7056743,-74.0123839,0.263530757401171,0,4.4,342,0.581586,4.0,143,0.858037,NaN,NaN,NaN,2,0.542714,0.719811,0.390651
29,29,Famous Calabria Pizza,27 St James Pl,40.7119418,-73.99934689999999,1.3158593427106806,0,4.5,291,0.905748,NaN,NaN,NaN,NaN,NaN,NaN,1,0.372414,0.905748,0.337313
28,28,Kesté Wall Street,77 Fulton St,40.7092446,-74.0048927,0.7799148904100284,0,4.5,539,0.905748,NaN,NaN,NaN,NaN,NaN,NaN,1,0.372414,0.905748,0.337313
25,25,Harry's Italian,2 Gold St,40.707555318913,-74.006898168681,0.5928193022318851,1,NaN,NaN,NaN,3.5,580,0.040859,8.1,512,1.170217,2,0.542714,0.605538,0.328634


In [34]:
# %%time
# # run all in parallel

# from multiprocessing import Pool

# services = ['gmaps', 'yelp', 'foursquare']
# args = list(product(services, [location_coords], [keyword]))

# def generic_get_df(service, location_coords, keyword):
#     """Query specified service for location and keyword, return dataframe"""

#     if service=="gmaps":
#         retdf = gmaps_get_df(location_coords, keyword)
#     elif service=="yelp":
#         retdf = yelp_get_df(location_coords, keyword)
#     elif service=="foursquare":
#         retdf = foursquare_get_df(location_coords, keyword)
        
#     return retdf

# def all_get_df(location_coords, keyword):
#     """Query all services for location and keyword, return list of dataframes"""

#     services = ['gmaps', 'yelp', 'foursquare']
#     args = list(product(services, [location_coords], [keyword]))
#     with Pool() as pool:
#         df_list = pool.starmap(generic_get_df, args)
#     df_list = [df for df in df_list if df is not None]    
#     return df_list

# df_list = all_get_df(location_coords, keyword) 
# dedupe_df =  dedupe(df_list)
# dedupe_df 

Importing data ...
Reading from dedupe_dataframe_learned_settings
Clustering...
# duplicate sets 54
CPU times: user 162 ms, sys: 220 ms, total: 382 ms
Wall time: 10.5 s


,cluster,name,address,lat,lng,distance,source,gmaps_rating,gmaps_nratings,gmaps_rating_std,yelp_rating,yelp_nratings,yelp_rating_std,foursquare_rating,foursquare_nratings,foursquare_rating_std,nratings,w,R,bayes_score
38,38,Scott's Pizza Tours,,40.750244140625,-73.9970245361328,4.9303850451248294,1,NaN,NaN,NaN,5.0,314,2.492392,NaN,NaN,NaN,1,0.372414,2.492392,0.928201
0,0,Joe’s Pizza,124 Fulton St,40.7101118,-74.0077386,0.5839478253839243,0,4.6,981,1.229911,4.0,200,0.858037,8.9,96,2.003460,3,0.640316,1.363802,0.873265
2,2,&pizza - Wall Street,63 Wall St,40.7057647,-74.0085474,0.5035979897455162,0,4.6,118,1.229911,4.5,81,1.675215,NaN,NaN,NaN,2,0.542714,1.452563,0.788325
4,4,Adrienne's Pizzabar,54 Stone St,40.7042941,-74.0100646,0.502332680670826,0,4.4,1636,0.581586,4.0,1656,0.858037,8.6,880,1.690994,3,0.640316,1.043539,0.668195
39,39,Jiannetto's Pizza Truck,100 Wall St,40.70525,-74.00675,0.6653736322792096,1,NaN,NaN,NaN,4.5,31,1.675215,NaN,NaN,NaN,1,0.372414,1.675215,0.623873
1,1,Big Al's Chicago Style Pizza,9 Thames St,40.7089116,-74.01159539999999,0.2345723937191282,0,4.6,150,1.229911,3.5,73,0.040859,7.7,35,0.753595,3,0.640316,0.674788,0.432078
6,6,The Grotto Pizzeria,69 New St,40.7056743,-74.0123839,0.263530757401171,0,4.4,342,0.581586,4.0,143,0.858037,NaN,NaN,NaN,2,0.542714,0.719811,0.390651
29,29,Famous Calabria Pizza,27 St James Pl,40.7119418,-73.99934689999999,1.3158593427106806,0,4.5,291,0.905748,NaN,NaN,NaN,NaN,NaN,NaN,1,0.372414,0.905748,0.337313
28,28,Kesté Wall Street,77 Fulton St,40.7092446,-74.0048927,0.7799148904100284,0,4.5,539,0.905748,NaN,NaN,NaN,NaN,NaN,NaN,1,0.372414,0.905748,0.337313
25,25,Harry's Italian,2 Gold St,40.707555318913,-74.006898168681,0.5928193022318851,1,NaN,NaN,NaN,3.5,580,0.040859,8.1,512,1.170217,2,0.542714,0.605538,0.328634


In [35]:
# locationdict = dict([('midtown', '40.7484, -73.9857'),
#                      ('downtown', '40.7077443,-74.0139089'),
#                      ('uppereastside', '40.7711473,-73.9661166'),
#                      ('upperwestside', '40.778794,-73.984257'),
#                      ('brooklynheights', '40.6915812,-73.9954095'), 
#                      ('grandarmyplaza', '40.671872,-73.972544'),
#                      ('bayridge', '40.6292633,-74.0309554'),
#                      ('williamsburg', '40.7144609,-73.9553373')])

# for k, l in product(['pizza', 'coffee', 'icecream'], 
#                     ['midtown','downtown','uppereastside','upperwestside','brooklynheights','grandarmyplaza','bayridge','williamsburg',]):
#     filename = "cache/%s_%s.pkl" % (k, l)
#     location_coords = eval(locationdict[l])
#     tempdf = pd.read_pickle(filename)
#     tempdf['distance'] = tempdf.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
#                                       axis=1)
#     tempdf.to_pickle(filename)
#     print(filename)

In [36]:
# # file to retrain with train_dedupe2.py
# can train better if we add the keyword, not yet implemented
# tempdf = None
# for k, l, j in product(['pizza', 'coffee', 'icecream'], 
#                     ['midtown','downtown','uppereastside','upperwestside','brooklynheights','grandarmyplaza','bayridge','williamsburg',],
#                     ['gmaps', 'yelp', 'foursquare']):
#     filename = "cache/%s_%s_%s.pkl" % (j, k, l)
#     try:
#         if tempdf is None:
#             tempdf = pd.read_pickle(filename)
#             tempdf['keyword']=k
#             print(filename)
#         else:
#             tempdf2 = pd.read_pickle(filename)
#             tempdf2['keyword']=k
#             tempdf = pd.concat([tempdf, tempdf2])
#             print(filename)
#     except:
#         print("missing ", filename)